In [ ]:
# !pip install -r ../requirements.txt

  Using cached patsy-1.0.1-py2.py3-none-any.whl.metadata (3.3 kB)
   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.8 MB 3.2 MB/s eta 0:00:03
   ----- ---------------------------------- 1.3/9.8 MB 3.3 MB/s eta 0:00:03
   -------- ------------------------------- 2.1/9.8 MB 3.3 MB/s eta 0:00:03
   ---------- ----------------------------- 2.6/9.8 MB 3.3 MB/s eta 0:00:03
   ------------- -------------------------- 3.4/9.8 MB 3.3 MB/s eta 0:00:02
   ----------------- ---------------------- 4.2/9.8 MB 3.3 MB/s eta 0:00:02
   ------------------- -------------------- 4.7/9.8 MB 3.3 MB/s eta 0:00:02
   ---------------------- ----------------- 5.5/9.8 MB 3.3 MB/s eta 0:00:02
   ------------------------- -------------- 6.3/9.8 MB 3.3 MB/s eta 0:00:02
   ---------------------------- ----------- 7.1/9.8 MB 3.3 MB/s eta 0:00:01
   ------------------------------ --------- 7.6/9.8 MB 3.3 MB/s eta 0:00:01
   --------------------------

# Imports

In [ ]:
import datetime

import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import xlrd

# Load Data

In [3]:
df = pd.read_excel("../data/Sample - Superstore.xls")

In [4]:
df.shape

(9994, 21)

In [5]:
df.isna().sum()

RowID           0
OrderID         0
OrderDate       0
ShipDate        0
ShipMode        0
CustomerID      0
CustomerName    0
Segment         0
Country         0
City            0
State           0
PostalCode      0
Region          0
ProductID       0
Category        0
Sub_Category    0
ProductName     0
Sales           0
Quantity        0
Discount        0
Profit          0
dtype: int64

In [18]:
df.dtypes

RowID                    int64
OrderID                 object
OrderDate       datetime64[ns]
ShipDate        datetime64[ns]
ShipMode                object
CustomerID              object
CustomerName            object
Segment                 object
Country                 object
City                    object
State                   object
PostalCode               int64
Region                  object
ProductID               object
Category                object
Sub_Category            object
ProductName             object
Sales                  float64
Quantity                 int64
Discount               float64
Profit                 float64
dtype: object

In [23]:
df["OrderDate"].dt.strftime('%Y-%m-%d')


0       2016-07-08
1       2016-04-03
2       2014-08-27
3       2016-04-03
4       2017-06-11
           ...    
9989    2014-03-07
9990    2016-11-10
9991    2014-03-07
9992    2014-03-07
9993    2014-02-18
Name: OrderDate, Length: 9994, dtype: object

In [21]:
print("Max Date:",df["OrderDate"].min().strftime('%Y-%m-%d'))
print("Min Date",df["OrderDate"].max().strftime('%Y-%m-%d'))

Max Date: 2014-01-03
Min Date 2017-12-30


In [7]:
print("number of product id",df["ProductID"].nunique())
print("number of product name",df["ProductName"].nunique())

number of product id 1862
number of product name 1850


In [8]:
result = (
    df[["ProductID", "ProductName"]]
    .groupby("ProductName")
    .agg(ProductID_count_distinct=("ProductID", "nunique"))  
    .sort_values("ProductID_count_distinct")
    .loc[lambda x: x["ProductID_count_distinct"] > 1]
    .reset_index()
)

print(result)


                                      ProductName  ProductID_count_distinct
0                         Avery Non-Stick Binders                         2
1                     Eldon Wave Desk Accessories                         2
2                        Prang Drawing Pencil Set                         2
3                         Storex Dura Pro Binders                         2
4   Peel & Seel Recycled Catalog Envelopes, Brown                         2
5                           Okidata C610n Printer                         2
6                      KI Adjustable-Height Table                         2
7                                   Staple magnet                         2
8         #10- 4 1/8" x 9 1/2" Recycled Envelopes                         2
9                      Staple-based wall hangings                         2
10                                  Staple holder                         3
11                                 Staple remover                         3
12          

In [ ]:
dict_agg = {
    "ProductID": "count",
    "Sales": "sum",
    "Quantity": "sum",
    "Profit": "sum",
    "Discount": "mean",
}

agg_df = (
    df[["ProductID", "OrderDate", "Sales", "Quantity", "Profit", "Discount"]]
    .groupby(["OrderDate"])
    .agg(dict_agg)
    .sort_values("OrderDate")
    .reset_index()
)

In [ ]:
agg_df.corr()

,OrderDate,ProductID,Sales,Quantity,Profit,Discount
OrderDate,1.000000,0.307168,0.170633,0.292095,0.089952,-0.002056
ProductID,0.307168,1.000000,0.622914,0.967554,0.224499,0.010172
Sales,0.170633,0.622914,1.000000,0.647291,0.489112,-0.044734
Quantity,0.292095,0.967554,0.647291,1.000000,0.224905,0.009359
Profit,0.089952,0.224499,0.489112,0.224905,1.000000,-0.228403
Discount,-0.002056,0.010172,-0.044734,0.009359,-0.228403,1.000000


In [ ]:
res = smf.ols("Profit ~ Discount", data=sm.add_constant(agg_df)).fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                 Profit   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     67.97
Date:                Sun, 22 Dec 2024   Prob (F-statistic):           4.20e-16
Time:                        23:47:01   Log-Likelihood:                -9830.7
No. Observations:                1237   AIC:                         1.967e+04
Df Residuals:                    1235   BIC:                         1.968e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    439.6964     31.884     13.791      0.0